In [ ]:
%matplotlib inline
import os, os.path
import matplotlib.pyplot as plt
import numpy as np
import firedrake
import icepack, icepack.plot, icepack.models

# Inverse problems

In this demo, we'll revisit the Larsen Ice Shelf.
This time, we're going to estimate the fluidity coefficient $A$ in Glen's flow law

$$\dot\varepsilon = A\tau^3$$

from observational data.
In the previous demos, we've come up with some value of the fluidity coefficient and computed a velocity field by solving an elliptic partial differential equation.
The fluidity coefficient is roughly a known function of the ice temperature, together with some fudge factors for crystal fabric or large-scale damage, so we know a rough range of values that it could take.
Nonetheless, we don't have large-scale measurements of the fluidity coefficient from remote sensing like we do for ice velocity and thickness.

Instead, we can try to come up with a value of $A$ that gives a velocity field closest to what we observed.
This rough idea can be turned into a constrained optimization problem.
The quantity we wish to optimize is the misfit between the computed velocity $u$ and the observed velocity $u^o$:

$$E(u) = \frac{1}{2}\int_\Omega\left(\frac{u - u^o}{\sigma}\right)^2dx,$$

where $\sigma$ are the standard deviations of the measurements.
In addition to minimizing the misfit, we also want to have a relatively smooth value of $A$.
The regularization functional $R$ is included to penalize oscillations over a given length scale $L$:

$$R(A) = \frac{L^2}{2}\int_\Omega|\nabla A|^2dx.$$

Finally, let $F(u, A)$ be the weak form of the shallow shelf equations; the constraint is that $F(u, A) = 0$.
We can enforce this constraint by introducing the Lagrange multiplier $\lambda$, in which case the combined objective functional is

$$J(u, A; \lambda) = E(u) + R(A) + \langle F(u, A), \lambda\rangle.$$

We can calculate the derivative of this functional with respect to $A$ by using the *adjoint method*.
We can then use a descent method to iterate towards a critical point, which is hopefully close to the true value of the fluidity coefficient.

### Input data

The input data are just as in the previous demo for the Larsen Ice Shelf, but we also need to use the error estimates for the velocities.

In [ ]:
data_directory = os.environ['ICEPACK_DATA']

In [ ]:
mesh = firedrake.Mesh(os.path.join(data_directory, "meshes/larsen/larsen.msh"))

fig, axes = icepack.plot.subplots()
axes.set_xlabel('meters')
axes.grid()
icepack.plot.triplot(mesh, axes=axes, linewidth=2)
plt.show(fig)

In [ ]:
from icepack.grid import arcinfo, GridData
thickness = arcinfo.read(os.path.join(data_directory, "bedmap2/larsen-h.txt"))
vx = arcinfo.read(os.path.join(data_directory, "measures_antarctica/larsen-vx.txt"))
vy = arcinfo.read(os.path.join(data_directory, "measures_antarctica/larsen-vy.txt"))
err = arcinfo.read(os.path.join(data_directory, "measures_antarctica/larsen-err.txt"))

In [ ]:
from preprocess import preprocess
vx = preprocess(vx, mesh)
vy = preprocess(vy, mesh)
thickness = preprocess(thickness, mesh, radius=6)

In [ ]:
degree = 2
Q = firedrake.FunctionSpace(mesh, 'CG', degree)
V = firedrake.VectorFunctionSpace(mesh, 'CG', degree)

h = icepack.interpolate(thickness, Q)
u_obs = icepack.interpolate(lambda x: (vx(x), vy(x)), V)
σ = icepack.interpolate(err, Q)

Here we'll plot the velocity errors.
You can see from the stripey pattern that they depend on the particular swath from the observational platform.

In [ ]:
fig, ax = icepack.plot.subplots()
contours = icepack.plot.tricontourf(σ, 20, axes=ax)
fig.colorbar(contours)
plt.show(fig)

We need to make an initial guess for the fluidity parameter.
In this case, we'll use the same value as in the second demo -- a constant fluidity assuming a temperature of $-13^\circ$C.

In [ ]:
T = 260
A = firedrake.interpolate(firedrake.Constant(icepack.rate_factor(T)), Q)

ice_shelf = icepack.models.IceShelf()
opts = {'dirichlet_ids': [3, 4, 5, 6, 7, 8], 'tol': 1e-6}
u = ice_shelf.diagnostic_solve(u0=u_obs, h=h, A=A, **opts)

### Inferring the fluidity

Solving an inverse problem has lots of moving parts, so we have an object called `InverseProblem` in icepack to manage everything for you.
Hopefully this class saves you from too many low-level details, but still provides a good amount of flexibility and transparency.
There are five parts that go into an inverse problem:
* a physics model
* an initial guess for the parameter and state
* an error metric
* a smoothness metric
* a priori bounds on the parameter

We already have the first two, so it remains to decide how we'll measure the misfit and smoothness.
Here we'll use the $L^2$-norm error for the velocities and the mean-square gradient for the fluidity, respectively.

In [ ]:
from firedrake import inner, grad, dx
import icepack.inverse

objective = 0.5 * (inner(u - u_obs, u - u_obs)) / σ**2 * dx

L = 5e3
regularization = 0.5 * L**2 * inner(grad(A), grad(A)) * dx

The ice temperature is certainly above $-25^\circ$C and below $0^\circ$C.
However, the actual value of the fluidity parameter might instead be consistent with a temperature above 0 in the case of ice that has experienced lots of damage from crevassing.
With that in mind, we'll take a maximum value for $A$ to be consistent with a temperature of $+5^\circ$C.

In [ ]:
Amin, Amax = icepack.rate_factor(248.0), icepack.rate_factor(278.0)
print(f'Min, max values of the fluidity: {Amin}, {Amax}')

As a convenience, the inverse problem object allows you to call a function of your choice at the end of every iteration.
For this demonstration, we'll have it print out the values of the misfit and regularization functionals.
You could also, say, make a plot of the state and parameter guess at every iteration to make a movie of how the algorithm progresses.

In [ ]:
area = firedrake.assemble(firedrake.Constant(1) * dx(mesh))
def print_error_and_regularization(inverse_problem):
    E = firedrake.assemble(inverse_problem.objective)
    R = firedrake.assemble(inverse_problem.regularization)
    B = firedrake.assemble(inverse_problem.barrier)
    print(f'{E/area:g}, {R/area:g}, {B/area:g}')

Next, we'll create the `InverseProblem` object.
We've already mentioned several objects that the inverse problem needs -- the model, the initial guess, some functionals, etc.
Additionally, it needs to know the name of the observed field and the parameter (the `state_name` and `parameter_name`) arguments, since these values are passed to the forward solver as keyword arguments.
All the additional arguments to the forward model are passed as a dictionary `model_args`.
In our case, these consist of the thickness field, the initial guess for the velocity, and the forward solver tolerance.
Finally, to specify the inverse problem, we need to know where Dirichlet boundary conditions are to be applied, as this affects how one solves for the Lagrange multiplier field $\lambda$.

Once we've created the problem, we then create a solver object that will iteratively search for a good value of the parameters.
The solver object is distinct from the problem so that it's easier to customize how the solver works.

In [ ]:
problem = icepack.inverse.InverseProblem(
    model=ice_shelf,
    method=icepack.models.IceShelf.diagnostic_solve,
    objective=objective,
    regularization=regularization,
    state_name='u',
    state=u,
    parameter_name='A',
    parameter=A,
    parameter_bounds=(Amin, Amax),
    barrier=1e-8,
    model_args={'h': h, 'u0': u, 'tol': 1e-6},
    dirichlet_ids=[3, 4, 5, 6, 7, 8]
)

solver = icepack.inverse.InverseSolver(problem, print_error_and_regularization)

The solve method takes in a relative convergence tolerance, an absolute tolerance, and a maximum number of iterations, and it returns the total number of iterations necessary to achieve the given tolerances.
In our case, we'll stop once the relative decrease in the objective function from one iteration to the next is less than 1/200.

The algorithm takes a few minutes to run.
Now would be the time to put on a fresh pot of coffee.

In [ ]:
iterations = solver.solve(
    rtol=5e-3,
    atol=0.0,
    max_iterations=30
)

### Analysis

Now that we're done, we'll want to to some post-processing and analysis on the fluidity parameter that we inferred.
The inverse problem object stores the parameter we're inferring and the observed field as the properties `parameter` and `state` respectively.
The names are intentionally not specific to just ice shelves.
For other problems, we might instead be inferring a friction coefficient rather than a fluidity, or we might be observing the thickness instead of the velocity.
You can see all the publicly visible properties by typing `help(inverse_problem)`.

In [ ]:
fig, ax = icepack.plot.subplots()
ctr = icepack.plot.tricontourf(solver.parameter, 40, axes=ax)
fig.colorbar(ctr)
plt.show(fig)

The fluidity is much higher around areas of heavy crevassing, and much lower around the northern edge of the ice shelf.
It's possible that some of the ice is actually grounded here, and the velocities are actually a result of basal drag.
In that case, the algorithm will erroneously give a low value of the fluidity, since that's the only variable at hand for explaining the observed velocities.

In [ ]:
fig, ax = icepack.plot.subplots()
ctr = icepack.plot.tricontourf(solver.state, 40, axes=ax)
fig.colorbar(ctr)
plt.show(fig)

The computed ice velocity is mostly similar to observations, but doesn't quite capture the sharp change in velocity near the big rift by Gipps Ice Rise.
We regularized the problem by looking only for smooth values of the fluidity parameter.
As a consequence, we won't be able to see sharp changes that might result from features like crevasses or rifts.
We might instead try to use the total variation functional

$$R(A) = L\int_\Omega|\nabla A|dx$$

if we were interested in features like this.

In [ ]:
fig, ax = icepack.plot.subplots()
ctr = icepack.plot.tricontourf(solver.adjoint_state, 40, axes=ax)
fig.colorbar(ctr)
plt.show(fig)

Plotting the adjoint state field directly can give us some idea of what areas of the ice shelf make the biggest difference for the total error.

Finally, let's try and see how different the inferred fluidity is from our naive guess:

In [ ]:
print(icepack.norm(solver.parameter - A) / icepack.norm(A))
print(firedrake.assemble(solver.objective) / area)

Our final approximation departs quite substantially from the initial guess.
This suggests that we can't get away with using simple guesses for making projections and that a data assimilation procedure is warranted.
Additionally, the value of the misfit functional divided by the ice shelf area should be around 1/2, provided that we had a good value of the fluidity parameter.
While the approximation we've found is an improvement over a naive guess, it leaves much to be desired still.

### Conclusion

In this demo, we've shown how to back out the fluidity of an ice shelf from remote sensing observations.
The value we find is not at all spatially homogeneous, so we probably couldn't have somehow parameterized it to get a reasonable guess.
We could then use this value, together with some sort of closure assumption for how the fluidity evolves, to initialize a prognostic model of the ice shelf.

We would expect from statistical estimation theory that the misfit functional divided by the shelf area will be around 1/2, since a sum of squared normal random variables has a $\chi^2$-distribution.
We are quite far off from that.
There are a number of reasons why this might happen.

* The error estimates $\sigma$ are wrong.
* We don't have a good way to also account for thickness errors, which are substantial.
* We regularized the problem too much.
* The ice shelf becomes grounded on some pinning point and we didn't add basal drag.
* I implemented the algorithm wrong.

It's probably a superposition of all of the above.
In any case, there's quite a bit of improvement possible on what we've shown here.